# **4: Hands-on Exploratory Data Analysis with DuckDB :duck:**

---

By Jean-Yves Tran | jy.tran@[datascience-jy.com](https://datascience-jy.com) | [LinkedIn](https://www.linkedin.com/in/jytran-datascience/)  
IBM Certified Data Analyst 

---

Source: 
- [Getting Started with DuckDB](https://www.packtpub.com/en-ar/product/getting-started-with-duckdb-9781803232539) by Simon Aubury & Ned Letcher
- [DuckDB documentation](https://duckdb.org/docs/)
---

The interactive links in this notebook are not working due to GitHub limitations. View this notebook with the interactive links working [here](https://nbviewer.org/github/jendives2000/Data_ML_Practice_2025/blob/main/1-3-SQL/practice/DuckDB/notebooks/4_duckdb_handson_eda.ipynb).

---

This is part 3 of this series of notebooks on DuckDB.  
For an introduction to DuckDB, check [my first notebook](https://github.com/jendives2000/Data_ML_Practice_2025/blob/82571ad44176666f9cf0735c5141c6a96d5eace9/1-3-SQL/practice/DuckDB/notebooks/1_duckdb_intro.ipynb). I also say in there when you should not use DuckDB. 
For understanding how DuckDB works, check my [second notebook](https://github.com/jendives2000/Data_ML_Practice_2025/blob/ef8533ad82586234cfdc54a494c0c5be590816cc/1-3-SQL/practice/DuckDB/notebooks/2_duckdb_python_API.ipynb).

Here I will learn about best practices that:
- save time when:
  - [querying](#select-) or [inserting](#insert) data to a DuckDB database
  - joining tables ([positional](#positional-joins) and [temporal joins](#temporal-joins-asof))

<u>**SqlMagic:**</u>  

You will notice starting from the insert chapter that I refactored a code that runs SQL commands from the DuckDB shell (CLI). I named it shell_commd(). 
There is a [better setup](https://duckdb.org/docs/guides/python/jupyter.html) for jupyter-based works that essentially:
- runs SQL cells in Jupyter (using just '%sql' before any query) 
- and delivers the output as a pandas dataframe. 

Such a setup requires to install and import jupysql, pandas of course and optionally matplotlib and duck-engine.  
I am **not using this setup** in this notebook but in the next one, which will be focused on data exploration and visualization. 

**OUTLINE:**  
In more details, I will cover the followings:
- [**Selecting columns**](#select-) effectively
- Applying [**function chaining**](#function-chaining)
- Using [**INSERT**](#insert) effectively
- Leveraging [**positional** joins](#positional-joins) and [**temporal joins**](#temporal-joins-asof)
- [**Recursive** queries](#hierarchical-traversal) and [macros](#macros)
- additional **tips and tricks**

**NICETIES:**  
Some of the nicest little improvements brought by DuckDB are: 
- [trailing comma](#trailing-comma-is-fine) not a problem
- the [exclude](#exclude-columns) clause
- [visual bars](#visualize-bars-in-the-output) in the output
- [insert or replace](#insert-or-replace)

**SKIERS DATABASE**:  
I added several very simple Skiers Database and csv files in the data/data_in folder: `skiers.csv`  
They'll be used throughout this notebook. 

**The main takeaway is**:
- to **better comprehend** the **differences** between regular SQL queries and DuckDB enhanced queries, which is often a lot less verbosity and more readability. 

---


## **Imports**:

In [5]:
#! pip install pandas matplotlib
import duckdb
import pandas as pd

## Dataset: Melbourne Pedestrian Count:

I will be using a dataset made available by the city of Melbourne, Australia. contains hourly pedestrian counts from pedestrian sensors located in and around the Melbourne Central business district. We’ll be working with a historical timeframe of this dataset ranging from 2009 to 2022.

I [imported](https://data.melbourne.vic.gov.au/api/datasets/1.0/pedestrian-counting-system-monthly-counts-per-hour/attachments/pedestrian_counting_system_monthly_counts_per_hour_may_2009_to_14_dec_2022_csv_zip/) it in the data/data_in folder. 
Once unzipped, this is:
- a 420MB dataset 
- with over 2,1 million entries


## Relational API: 

I will use the Relational API because it offers more for what I will be doing: data exploratory analysis. 

Let's get our dataset into a Relational Object (RO from now on):

In [6]:
records = duckdb.read_csv("../data/data_in/pedestrian_records_2009-2022.csv")

# 200 is the max number of characters possible inside an entry:
records.show(max_width = 200)

┌─────────┬───────────────────────────────┬───────┬──────────┬───────┬──────────┬───────┬───────────┬───────────────────────────────┬───────────────┐
│   ID    │           Date_Time           │ Year  │  Month   │ Mdate │   Day    │ Time  │ Sensor_ID │          Sensor_Name          │ Hourly_Counts │
│  int64  │            varchar            │ int64 │ varchar  │ int64 │ varchar  │ int64 │   int64   │            varchar            │     int64     │
├─────────┼───────────────────────────────┼───────┼──────────┼───────┼──────────┼───────┼───────────┼───────────────────────────────┼───────────────┤
│ 2887628 │ November 01, 2019 05:00:00 PM │  2019 │ November │     1 │ Friday   │    17 │        34 │ Flinders St-Spark La          │           300 │
│ 2887629 │ November 01, 2019 05:00:00 PM │  2019 │ November │     1 │ Friday   │    17 │        39 │ Alfred Place                  │           604 │
│ 2887630 │ November 01, 2019 05:00:00 PM │  2019 │ November │     1 │ Friday   │    17 │        37 

## **Looking at the Data**:

Because this is an RO, DuckDB loaded 10,000 rows and this what it **lazily returned** us here. There more entries than these 10,000. 

Lots of info in this dataset: 
- **count of pedestrians** detected by 
- a specific **sensor** 
- during **each hour**. 
- Additionally, it provides other details related to the hourly readings, including the **sensor name** 
  - and the **timestamp**, along with date and time components derived from the timestamp.

### **Data types**:
Notice in the header of the output that datetime is of the data type VARCHAR, meaning text. It should be a timestamp (docs on this [here](http://duckdb.org/docs/sql/functions/dateformat)). Let's fix that:

In [7]:
records = duckdb.read_csv(
    "../data/data_in/pedestrian_records_2009-2022.csv",
    dtype={"Date_Time": "TIMESTAMP"},
    timestamp_format="%B %d, %Y %H:%M:%S %p",
)

Let's confirm this change:

In [8]:
records.limit(5).show(max_width=200)

┌─────────┬─────────────────────┬───────┬──────────┬───────┬─────────┬───────┬───────────┬──────────────────────────────┬───────────────┐
│   ID    │      Date_Time      │ Year  │  Month   │ Mdate │   Day   │ Time  │ Sensor_ID │         Sensor_Name          │ Hourly_Counts │
│  int64  │      timestamp      │ int64 │ varchar  │ int64 │ varchar │ int64 │   int64   │           varchar            │     int64     │
├─────────┼─────────────────────┼───────┼──────────┼───────┼─────────┼───────┼───────────┼──────────────────────────────┼───────────────┤
│ 2887628 │ 2019-11-01 17:00:00 │  2019 │ November │     1 │ Friday  │    17 │        34 │ Flinders St-Spark La         │           300 │
│ 2887629 │ 2019-11-01 17:00:00 │  2019 │ November │     1 │ Friday  │    17 │        39 │ Alfred Place                 │           604 │
│ 2887630 │ 2019-11-01 17:00:00 │  2019 │ November │     1 │ Friday  │    17 │        37 │ Lygon St (East)              │           216 │
│ 2887631 │ 2019-11-01 17:00:00 │ 

### **Enums for low cardinality String Columns**:

[ENUM types](https://duckdb.org/docs/sql/data_types/enum.html) are a way to **convert string values into numbers** in a database. This is useful for columns with a limited number of different values, like month names and days of the week. By using ENUMs for these columns, we can **save storage space** and **speed up queries** because the database only stores numbers instead of full strings.  

In DuckDB, because the **SQL parser doesn’t support subqueries** in a CREATE TYPE ... AS ENUM statement, the best approach is to **extract** the distinct values via SQL (or pandas), **format** them into a comma-separated list, and then **build** the ENUM type dynamically.  
This method gives me flexibility and control over the ENUM values while keeping my code concise.

In [ ]:
# refactored logic to create ENUM with an RO:
def create_enum(col, db, enum_name):
    """
    Create an ENUM type in DuckDB from unique values in a specified column.

    Parameters:
    col (str): The column name to extract unique values from.
    db (str): The name of the DuckDB relation (table).
    enum_name (str): The name of the ENUM type to be created.
    """
    # extracting the unique values from the specified column
    col_forEnum = duckdb.sql(
        f"""
        select distinct {col}
        from {db}
        """
    ).fetchall()

    # Build a comma-separated list of ENUM values
    col_enum_val = ", ".join(f"'{v[0]}'" for v in col_forEnum)

    # create the ENUM type
    duckdb.sql(f"create type {enum_name} as enum ({col_enum_val});")

create_enum("Month", "records", "month_enum")

In [10]:
create_enum("Day", "records", "day_enum")

Did I actually created them? 

In [12]:
print(duckdb.sql("select enum_range(NULL::month_enum);"))
print(duckdb.sql("select enum_range(NULL::day_enum);"))

┌────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                enum_range(CAST(NULL AS month_enum))                                │
│                                             varchar[]                                              │
├────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ [October, April, July, January, March, August, September, December, May, June, February, November] │
└────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────────┐
│                enum_range(CAST(NULL AS day_enum))                │
│                            varchar[]                             │
├──────────────────────────────────────────────────────────────────┤
│ [Thursday, Wednesday, Monday, Friday, Saturday, Tuesday, Sunday] │
└───────────────────────────────────

Before we load our dataset into an on-disk database for analysis, we should think about any data changes we might want to make. For instance, we can remove columns we won’t need. The additional date and time fields are likely to be useful, and we should keep the Sensor_ID column since it helps connect this dataset with another one from the city of Melbourne that includes details about each sensor, like their locations. However, we can drop the ID field because it doesn’t relate to any other datasets about the pedestrian counting system. Since we’ll be doing various time series analyses, we also need to sort the records by the Date_Time column. Let’s make these changes and take a look at the results.